In [1]:
import pandas as pd
import ta #Tecnical indicators
from binance import Client
from datetime import timedelta # Add and substract timestamps
import time
from pytz import timezone
from keys import api_key, api_secret
client = Client(api_key, api_secret)

In [2]:
symbol = 'ETHUSDT' #Create symbols list for iteration

In [3]:
def getdata(symbol):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                     '15m', # original '15m'
                                                      '3300 minutes UTC')) # original '3000'
    frame = frame.iloc[:,0:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame.set_index('Time', inplace=True)
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame.index = frame.index.tz_localize('UTC')
    utc_minus_3 = timezone('Etc/GMT+3')
    frame.index = frame.index.tz_convert(utc_minus_3)
    frame.index = frame.index.strftime('%Y-%m-%d %H:%M')
    frame= frame.astype(float)
    return frame

In [281]:
df = getdata(symbol)

In [282]:
new_df = df

In [283]:
new_df

,Open,High,Low,Close
Time,,,,
2023-06-26 18:00,1851.44,1855.49,1849.61,1854.89
2023-06-26 18:15,1854.88,1856.44,1852.60,1854.15
2023-06-26 18:30,1854.16,1854.45,1851.41,1851.42
2023-06-26 18:45,1851.41,1852.51,1847.45,1850.80
2023-06-26 19:00,1850.80,1854.37,1847.02,1853.70
...,...,...,...,...
2023-06-28 23:45,1833.27,1834.50,1831.51,1832.55
2023-06-29 00:00,1832.56,1836.15,1831.25,1835.19
2023-06-29 00:15,1835.19,1837.85,1833.69,1837.65


In [284]:
new_df['stochrsi_k'] = ta.momentum.stochrsi_k(new_df.Close, window=10, smooth1=3, smooth2=3)
new_df['stochrsi_k'] = new_df['stochrsi_k'].round(2)
new_df['Buy']=(new_df.stochrsi_k[-2] <= 0.049) & (new_df.stochrsi_k[-1] > 0.049) 
new_df['Sell']=(new_df.stochrsi_k[-2] >= 0.90) & (new_df.stochrsi_k[-1] < 0.899)

In [285]:
new_df

,Open,High,Low,Close,stochrsi_k,Buy,Sell
Time,,,,,,,
2023-06-26 18:00,1851.44,1855.49,1849.61,1854.89,NaN,False,False
2023-06-26 18:15,1854.88,1856.44,1852.60,1854.15,NaN,False,False
2023-06-26 18:30,1854.16,1854.45,1851.41,1851.42,NaN,False,False
2023-06-26 18:45,1851.41,1852.51,1847.45,1850.80,NaN,False,False
2023-06-26 19:00,1850.80,1854.37,1847.02,1853.70,NaN,False,False
...,...,...,...,...,...,...,...
2023-06-28 23:45,1833.27,1834.50,1831.51,1832.55,0.14,False,False
2023-06-29 00:00,1832.56,1836.15,1831.25,1835.19,0.15,False,False
2023-06-29 00:15,1835.19,1837.85,1833.69,1837.65,0.41,False,False


In [290]:
new_df = new_df[-200:]

In [291]:
new_df_copy = new_df.copy()
new_df_copy = new_df_copy[-1:]

In [292]:
new_df_copy

,Open,High,Low,Close,stochrsi_k,Buy,Sell
Time,,,,,,,
2023-06-29 00:45,1836.41,1836.76,1835.81,1836.76,0.72,False,False


In [293]:
new_df_copy.to_csv('historic_new_value.csv',index_label=False)

In [294]:
#load cvs to update
historic_df = pd.read_csv('historic_200_values.csv')
#load cvs new value
new_value_df = pd.read_csv('historic_new_value.csv')

In [295]:
new_value_df

,Open,High,Low,Close,stochrsi_k,Buy,Sell
2023-06-29 00:45,1836.41,1836.76,1835.81,1836.76,0.72,False,False


In [296]:
historic_df

,Open,High,Low,Close,stochrsi_k,Buy,Sell
2023-06-26 23:00,1864.03,1864.42,1862.00,1863.83,0.76,False,False
2023-06-26 23:15,1863.84,1866.40,1863.22,1864.78,0.67,False,False
2023-06-26 23:30,1864.77,1868.02,1864.77,1868.01,0.77,False,False
2023-06-26 23:45,1868.02,1876.71,1867.10,1875.60,0.91,False,False
2023-06-27 00:00,1875.60,1878.93,1871.63,1872.74,0.86,False,False
...,...,...,...,...,...,...,...
2023-06-28 23:45,1833.27,1834.50,1831.51,1832.55,0.14,False,False
2023-06-29 00:00,1832.56,1836.15,1831.25,1835.19,0.15,False,False
2023-06-29 00:15,1835.19,1837.85,1833.69,1837.65,0.41,False,False
2023-06-29 00:30,1837.65,1837.65,1835.33,1836.41,0.60,False,False


In [297]:
#Concat new data
updated_df = pd.concat([historic_df, new_value_df])
#slice dataframe to size 200 units for charts
updated_df = updated_df[-200:]
updated_df.to_csv('historic_200_values.csv', index_label=False)
updated_df.to_csv('historic_CHART_values.csv', index_label='Time', index=True)
updated_df

,Open,High,Low,Close,stochrsi_k,Buy,Sell
2023-06-26 23:15,1863.84,1866.40,1863.22,1864.78,0.67,False,False
2023-06-26 23:30,1864.77,1868.02,1864.77,1868.01,0.77,False,False
2023-06-26 23:45,1868.02,1876.71,1867.10,1875.60,0.91,False,False
2023-06-27 00:00,1875.60,1878.93,1871.63,1872.74,0.86,False,False
2023-06-27 00:15,1872.75,1873.41,1870.64,1873.41,0.74,False,False
...,...,...,...,...,...,...,...
2023-06-29 00:00,1832.56,1836.15,1831.25,1835.19,0.15,False,False
2023-06-29 00:15,1835.19,1837.85,1833.69,1837.65,0.41,False,False
2023-06-29 00:30,1837.65,1837.65,1835.33,1836.41,0.60,False,False
2023-06-29 00:45,1836.41,1836.76,1835.81,1836.76,0.72,False,False


In [289]:
new_df.to_csv('historic_200_values.csv', index_label=False)

In [288]:
new_df.to_csv('historic_CHART_values.csv', index=True)

In [ ]:
def indicators(df):
    df['stochrsi_k'] = ta.momentum.stochrsi_k(df.Close, window=10, smooth1=3, smooth2=3)
    df.drop(df.index[:-6], inplace=True)
    df['Buy']= (df.stochrsi_k[-2] <= 0.049) & (df.stochrsi_k[-1] > 0.049) 
    df['Sell']= (df.stochrsi_k[-2] >= 0.90) & (df.stochrsi_k[-1] < 0.899)
    return df

In [ ]:
indicators(df)

In [ ]:
str(df['stochrsi_k'][-1])[:4]

In [ ]:
def pricecalc(symbol, limit=1): #CHANGE PRICE (97% OK?)
    raw_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
    dec_len= len(str(raw_price).split('.')[1])
    price = raw_price * limit
    return round(price, dec_len)

def quantitycalc(symbol, investment):
    info = client.get_symbol_info(symbol=symbol)
    Lotsize = float([i for i in info['filters'] if
                    i['filterType'] == 'LOT_SIZE'][0]['minQty'])
    price = pricecalc(symbol)
    qty = round(investment/price, right_rounding(Lotsize))
    return qty

def right_rounding(Lotsize):
    splitted = str(Lotsize).split('.')
    if float(splitted[0]) == 1:
        return 0
    else:
        return len(splitted[1])

In [ ]:
pos_dict = {'in_position': False}
test_price = 0 #only for test

In [ ]:
def buy(investment): # Add symbol variable for iteration
    # Use test buy order
    #order = client.order_limit_buy(symbol=symbol,
                                 # price = pricecalc(symbol),
                                # quantity = quantitycalc(symbol, investment))
    order = client.create_test_order(symbol=symbol,side='BUY',type='LIMIT',
                                  price = pricecalc(symbol), timeInForce='GTC',
                                  quantity = quantitycalc(symbol, investment))
    
    print(order)
    print('BUY order placed',symbol,df['Close'][-1],str(df['stochrsi_k'][-1])[:4],str(df.index[-1])[0:16]) #Only for test
    pos_dict['in_position'] = True
    test_price = df['Close'][-1] # only for test
    return order

def sell(qty):
    #order = client.create_order(symbol=symbol,
                               #side= 'SELL',
                               #type= 'MARKET',
                               #quantity=qty)
    order = client.create_test_order(symbol=symbol,
                               side= 'SELL',
                               type= 'MARKET',
                               quantity=qty)
    print(order)
    print('SELL order placed',symbol,df['Close'][-1],str(df['stochrsi_k'][-1])[:4],str(df.index[-1])[0:16]) #Only for test
    pos_dict['in_position']= False

In [ ]:
def checkbuy(investment):
    if not pos_dict['in_position']:
        if df.Buy.values[-1]:
            return True
        else:
            print('Not a Buy signal') # Not final, doubt
    else:
        print('already in a position')

In [ ]:
def checksell(): # Add symbol variable for iteration
    
    #order_status = client.get_order(symbol= symbol, orderId=order['orderId']) for REAL
    
    #if pos_dict['in_position']:                                    for REAL
        #if order_status['status'] == 'NEW':                        for REAL
            #print('buy limit= still order pending')
        #elif order_status['status'] =='FILLED':
            # Check for 1% loss
            #cond1 = df.Close.values[-1] <= float(order_status['price'])*0.99
            # Check for Sell indicator
            #cond2 = df.Sell.values[-1] == True
            #if cond1 or cond2:
                #sell(order_status['origQty'])                          for REAL
    #else:
        #print('currently not in position, no checks for selling') for REAL
    
    if pos_dict['in_position']:
        # Check for 1% loss
        cond1 = df.Close.values[-1] <= test_price * 0.99 
        # Check for Sell indicator
        cond2 = df.Sell.values[-1] == True
        if cond1 or cond2:
            sell(100) #value for test
        else:
            print('No Sell conditions found') # Not final, doubt
    else:
        print('currently not in position, no checks for selling')

In [ ]:
investment = 100 # client.get_asset_balance(asset='USDT') index dictionary for balance
# For running this script you should work with a "cron job"
# and not a "while loop". Cron jobs examples in other bot
# videos
while True: #Add iteration for symbols "for i in symbols(create list)"
    df = indicators(getdata(symbol))
    if checkbuy(investment):
        curr_order = buy(investment)
    try:
        #checksell(curr_order) For real not TEST
        checksell()
    except:
        print('Not an order yet')
    time.sleep(900) #60 secconds for test 15min (900) for execute